In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Path where the CSV files are located
data_dir = 'C:/Users/ASUS/OneDrive/Desktop/GAMEEMO/Preprocessed EEG Data/.csv format/'

# Read and merge all the CSV files, and calculate the mean for each signal and group
for i in range(1, 28 + 1):  # Loop for signals S01 to S28
    for group_number in range(1, 4 + 1):  # Loop for groups G1 to G4
        signal = f"S{i:02}"
        group = f"G{group_number}"
        file_path = f'final_mean_{data_dir}{signal}.csv'

        # Read the data
        data = pd.read_csv(file_path)

        # Calculate mean values for all columns (excluding the label column, if exists)
        mean_values = data.mean()

        # Save the mean values to a new CSV file
        mean_values.to_csv(f'{data_dir}mean_{signal}{group}.csv', header=['Mean'])
        print(f"Mean values for {signal}{group} have been saved to 'mean_{signal}{group}.csv'")

# Now, let's create a dataset for deep learning with the mean values
# Assuming you've merged or created the dataset with `mean_{signal}{group}.csv` files.
mean_data = []

for i in range(1, 28 + 1):
    for group_number in range(1, 4 + 1):
        signal = f"S{i:02}"
        group = f"G{group_number}"
        file_path = f'{data_dir}mean_{signal}{group}.csv'

        # Read the mean values
        mean_values = pd.read_csv(file_path, index_col=0)

        # Add the mean values and label to the dataset
        mean_data.append(mean_values)

# Convert list to DataFrame
mean_df = pd.concat(mean_data, axis=1).T  # Transpose so signals are rows, features are columns
mean_df = mean_df.reset_index(drop=True)  # Reset index for clean dataframe

# Example: Assuming the last column is the label (mood), and the rest are features
X = mean_df.drop('Mood', axis=1).values  # Features (drop the label column)
y = mean_df['Mood'].values  # Labels (e.g., "happy", "sad", "normal")

# Encode the labels (mood categories) into numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape the data for LSTM input (3D array: [samples, time_steps, features])
time_steps = 256  # Update this based on your data

# Check if the number of features is divisible by time_steps
if X.shape[1] % time_steps != 0:
    raise ValueError(f"Cannot reshape array with shape {X.shape} into (time_steps, features) because {X.shape[1]} is not divisible by {time_steps}")

features = X.shape[1] // time_steps  # Features per time step

# Reshape X into 3D array (samples, time_steps, features)
X = X.reshape(-1, time_steps, features)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()

# LSTM Layer
model.add(LSTM(units=128, return_sequences=False, input_shape=(time_steps, features)))

# Dropout to prevent overfitting
model.add(Dropout(0.2))

# Fully connected layer
model.add(Dense(units=64, activation='relu'))

# Output layer with softmax activation for multi-class classification
model.add(Dense(units=3, activation='softmax'))  # 3 classes: happy, sad, normal

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

OSError: [Errno 22] Invalid argument: 'final_mean_C:/Users/ASUS/OneDrive/Desktop/GAMEEMO/Preprocessed EEG Data/.csv format/S01.csv'

Finding the mood using the csv

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os
import glob
import matplotlib.pyplot as plt

# Step 1: Combine All CSV Files for Each Signal and Group
def combine_csv_files(base_path):
    final_data_frames = []
    labels = []  # To store labels for each file (mood categories: Happy, Sad, Normal)
    
    for i in range(1, 28 + 1):  # Iterate over signals S01 to S28
        signal = f"S{i:02}"
        mean_data_frames = []
        
        for group_number in range(1, 4 + 1):  # Iterate over groups G1 to G4
            group = f"G{group_number}"
            mean_file_path = f'{base_path}({signal})/Preprocessed EEG Data/.csv format/mean_{signal}{group}.csv'
            
            try:
                # Read the mean CSV file
                mean_data = pd.read_csv(mean_file_path, index_col=0)
                mean_data_frames.append(mean_data)
            except FileNotFoundError:
                print(f"File not found: {mean_file_path}")
                continue

        if mean_data_frames:
            # Combine group means and calculate final mean for the signal
            combined_means = pd.concat(mean_data_frames, axis=1)  # Combine group means horizontally
            final_mean = combined_means.mean(axis=1)  # Calculate mean across groups
            
            # Add signal's final mean to the dataset
            final_data_frames.append(final_mean)

            # Manually assign mood labels for now (replace with actual mood mapping logic)
            # Example: 0 = Happy, 1 = Sad, 2 = Normal
            labels.append(np.random.randint(0, 3))  # Replace with your mood labels
    
    # Combine all signals' final means into a single DataFrame
    if final_data_frames:
        final_dataset = pd.concat(final_data_frames, axis=1).T.reset_index(drop=True)
        final_dataset['Mood'] = labels  # Add the mood labels as the target column
        return final_dataset
    else:
        raise Exception("No valid mean data found to process.")

# Step 2: Load and Prepare the Data
base_path = 'C:/Users/ASUS/OneDrive/Desktop/GAMEEMO/'  # Update to your base path
data = combine_csv_files(base_path)

# Split into features and labels
X = data.iloc[:, :-1].values  # Features (EEG mean values)
y = data.iloc[:, -1].values   # Labels (Mood categories)

# Encode labels (0 = Happy, 1 = Sad, 2 = Normal)
y = to_categorical(y)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape features for LSTM input (samples, timesteps, features)
time_steps = 256  # Update this based on your data
features = X.shape[1] // time_steps
X = X.reshape(-1, time_steps, features)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Build the LSTM Model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(y.shape[1], activation='softmax')  # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32)

# Step 4: Evaluate the Model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Save the model
model.save("eeg_mood_lstm_model.h5")

# Step 5: Visualize Training Performance
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()